In [97]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import csv
import json
import numpy as np

In [98]:
# Read CSV into dataframe
shelterCSV = "Resources/aac_intakes_outcomes.csv"
shelter_df = pd.read_csv(shelterCSV)

In [99]:
# Read JSON into dataframe
# breedJSON = "Resources/breed_info_2.json"
with open('Resources/breed_info.json', 'r') as f:
    pets_attr_dict = json.load(f)
    
    

dogs_attr_dict = pets_attr_dict["dog_breeds"]
dogs_attr_dict

dogs_df = pd.DataFrame.from_dict(dogs_attr_dict)
dogs_df = dogs_df.transpose()
dogs_df = dogs_df.reset_index()
dogs_df.rename(columns={'index':'Breed'}, inplace=True)

# Rename Columns
dogs_df = dogs_df.rename(columns={" Adaptability":"Adaptability"," All Around Friendliness":"All Around Friendliness"})
dogs_df = dogs_df.sort_values('Breed', ascending=True)
dogs_df.head()

,Breed,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,...,Potential For Playfulness,Potential For Weight Gain,Prey Drive,Sensitivity Level,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential
0,Affenpinscher,3.0,3.0,4.0,2.0,3.0,5.0,5.0,1.0,4.0,...,4.0,3.0,3.0,3.0,1.0,2.0,1.0,3.0,3.0,2.0
1,Afghan Hound,4.0,4.0,4.0,2.0,3.0,5.0,5.0,4.0,4.0,...,4.0,1.0,5.0,5.0,4.0,2.0,2.0,5.0,5.0,5.0
2,Airedale Terrier,2.0,4.0,5.0,3.0,5.0,1.0,4.0,2.0,4.0,...,5.0,4.0,5.0,3.0,3.0,4.0,2.0,3.0,3.0,4.0
3,Akita,3.0,2.0,4.0,4.0,4.0,2.0,5.0,5.0,1.0,...,5.0,4.0,4.0,5.0,4.0,5.0,1.0,5.0,2.0,4.0
4,Alaskan Klee Kai,3.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,2.0,...,4.0,2.0,5.0,4.0,2.0,3.0,2.0,5.0,2.0,4.0


In [100]:
# Make List for Column Categories and Copy into dataframe
shelter_cols = ["breed","animal_type","age_upon_outcome_(days)","outcome_type","sex_upon_outcome","time_in_shelter_days"]
Shelter_df = shelter_df[shelter_cols].copy()

# Drop Data That Isn't Related to Dogs
Shelter_df = Shelter_df[Shelter_df.animal_type == 'Dog']


# Rename Columns
Shelter_df = Shelter_df.rename(columns={"age_upon_outcome_(days)":"outcome_age_days",
                                        "sex_upon_outcome":"gender"})

# Added more specific age of intake using exact 'time_in_shelter_days' 
Shelter_df['intake_age_days'] = Shelter_df['outcome_age_days'] - Shelter_df['time_in_shelter_days']

# Cleaning Up Breeds
Shelter_df["Breed"] = Shelter_df["breed"].str.split('/').str[0]
Shelter_df["Sub"] = Shelter_df["breed"].str.split('/').str[-1]

# Dropped unwanted columns
Shelter_df = Shelter_df.drop(['animal_type', 'breed', 'Sub'], axis=1)

Shelter_df = Shelter_df.astype({"outcome_age_days": float})
Shelter_df.head()

,outcome_age_days,outcome_type,gender,time_in_shelter_days,intake_age_days,Breed
0,3650.0,Return to Owner,Neutered Male,0.588194,3649.411806,Spinone Italiano Mix
1,2555.0,Return to Owner,Neutered Male,1.259722,2553.740278,Spinone Italiano Mix
2,2190.0,Return to Owner,Neutered Male,1.113889,2188.886111,Spinone Italiano Mix
3,3650.0,Transfer,Neutered Male,4.970139,3645.029861,Dachshund
4,5840.0,Return to Owner,Neutered Male,0.119444,5839.880556,Shetland Sheepdog


In [101]:
group = Shelter_df.groupby('Breed')
group
time_in_shelter_days = group['time_in_shelter_days'].mean()
outcome_age_days = group['outcome_age_days'].mean()
intake_age_days = group['intake_age_days'].mean()

time_in_df = pd.DataFrame(time_in_shelter_days)
outcome_age_df = pd.DataFrame(outcome_age_days)
intake_age_df = pd.DataFrame(intake_age_days)

shelter_details_df=pd.concat([time_in_df,outcome_age_df,intake_age_df], axis=1)
shelter_details_df.head()

,time_in_shelter_days,outcome_age_days,intake_age_days
Breed,,,
Affenpinscher Mix,2.334028,886.428571,884.094544
Afghan Hound,46.824306,730.000000,683.175694
Afghan Hound Mix,1.140972,730.000000,728.859028
Airedale Terrier,12.259375,1039.166667,1026.907292
Airedale Terrier Mix,13.043704,1039.666667,1026.622963


In [102]:
######## This is a mess and I need help formatting it correctly
### JSON Data and Target Columns
breed_cols = ["Affectionate with Family","Amount of Shedding","Friendly Toward Strangers",
              "Intelligence","Potential For Playfulness"]

all_dog_data_df = pd.merge(dogs_df, shelter_details_df, on=["Breed"], how='outer')
all_dog_data_df.count()
all_dog_data_df = all_dog_data_df.dropna(how='any')
all_dog_data_df
# dog_data_counts = all_dog_data_df["Breed"].value_counts().sort_index(ascending=False).sort_values(ascending=False)
# dog_data_counts



,Breed,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,...,Sensitivity Level,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential,time_in_shelter_days,outcome_age_days,intake_age_days
1,Afghan Hound,4.0,4.0,4.0,2.0,3.0,5.0,5.0,4.0,4.0,...,5.0,4.0,2.0,2.0,5.0,5.0,5.0,46.824306,730.000000,683.175694
2,Airedale Terrier,2.0,4.0,5.0,3.0,5.0,1.0,4.0,2.0,4.0,...,3.0,3.0,4.0,2.0,3.0,3.0,4.0,12.259375,1039.166667,1026.907292
3,Akita,3.0,2.0,4.0,4.0,4.0,2.0,5.0,5.0,1.0,...,5.0,4.0,5.0,1.0,5.0,2.0,4.0,13.203980,1357.692308,1344.488328
5,Alaskan Malamute,2.0,4.0,5.0,3.0,4.0,1.0,4.0,5.0,2.0,...,4.0,4.0,5.0,1.0,5.0,2.0,5.0,6.107824,1161.666667,1155.558843
6,American Bulldog,2.0,3.0,4.0,3.0,3.0,1.0,5.0,1.0,2.0,...,4.0,4.0,2.0,1.0,2.0,2.0,3.0,31.551841,979.920635,948.368794
9,American Foxhound,2.0,5.0,5.0,4.0,4.0,1.0,4.0,3.0,5.0,...,2.0,3.0,5.0,1.0,4.0,4.0,5.0,9.147743,311.250000,302.102257
10,American Pit Bull Terrier,3.0,4.0,5.0,3.0,4.0,3.0,5.0,4.0,1.0,...,4.0,3.0,3.0,1.0,2.0,3.0,4.0,37.339782,1019.047619,981.707837
11,American Staffordshire Terrier,2.0,4.0,4.0,3.0,4.0,2.0,4.0,2.0,2.0,...,3.0,3.0,4.0,2.0,2.0,3.0,3.0,28.455378,857.659574,829.204196
15,Australian Cattle Dog,3.0,4.0,5.0,3.0,4.0,2.0,5.0,3.0,4.0,...,4.0,3.0,2.0,3.0,4.0,4.0,4.0,20.816758,924.518950,903.702193
16,Australian Kelpie,3.0,3.0,4.0,3.0,5.0,2.0,4.0,3.0,3.0,...,4.0,3.0,4.0,2.0,3.0,5.0,4.0,21.614043,786.269841,764.655798


In [103]:
all_breeds = list(all_dog_data_df["Breed"])
all_percentages = []

for breed in all_breeds:
    breed_df = Shelter_df.loc[Shelter_df['Breed'] == breed]
    number_adopted = (breed_df['outcome_type'] == "Adoption").sum()
    returned_to_owner = (breed_df['outcome_type'] == "Return to Owner").sum()
    total_outcomes = breed_df['outcome_type'].count()
    total_good_outcomes = number_adopted+returned_to_owner
    pct_good_outcomes = total_good_outcomes/total_outcomes
    pct_good_outcomes = "{0:.0%}".format(pct_good_outcomes)
    all_percentages.append(pct_good_outcomes)
#     good_outcome_pct_dict.update({breed:pct_good_outcomes})
    
good_outcome_pct_dict = {"Breed":all_breeds,"Good Outcome Percentage":all_percentages}
good_outcome_pct_dict

good_outcome_df = pd.DataFrame(good_outcome_pct_dict)
good_outcome_df

,Breed,Good Outcome Percentage
0,Afghan Hound,100%
1,Airedale Terrier,67%
2,Akita,69%
3,Alaskan Malamute,80%
4,American Bulldog,83%
5,American Foxhound,75%
6,American Pit Bull Terrier,76%
7,American Staffordshire Terrier,60%
8,Australian Cattle Dog,79%
9,Australian Kelpie,79%


In [105]:
final_dog_data_df = pd.merge(all_dog_data_df, good_outcome_df)
final_dog_data_df = final_dog_data_df.rename(columns={"time_in_shelter_days":"Avg Days In Shelter",
                                                  "outcome_age_days":"Avg Age Upon Outcome",
                                                  "intake_age_days":"Avg Age On Intake"})
final_dog_data_df
# headers = list(all_dog_data_df.columns.values)
# headers

,Breed,Adaptability,All Around Friendliness,Exercise Needs,Health Grooming,Trainability,Adapts Well to Apartment Living,Affectionate with Family,Amount Of Shedding,Dog Friendly,...,Size,Tendency To Bark Or Howl,Tolerates Being Alone,Tolerates Cold Weather,Tolerates Hot Weather,Wanderlust Potential,Avg Days In Shelter,Avg Age Upon Outcome,Avg Age On Intake,Good Outcome Percentage
0,Afghan Hound,4.0,4.0,4.0,2.0,3.0,5.0,5.0,4.0,4.0,...,4.0,2.0,2.0,5.0,5.0,5.0,46.824306,730.000000,683.175694,100%
1,Airedale Terrier,2.0,4.0,5.0,3.0,5.0,1.0,4.0,2.0,4.0,...,3.0,4.0,2.0,3.0,3.0,4.0,12.259375,1039.166667,1026.907292,67%
2,Akita,3.0,2.0,4.0,4.0,4.0,2.0,5.0,5.0,1.0,...,4.0,5.0,1.0,5.0,2.0,4.0,13.203980,1357.692308,1344.488328,69%
3,Alaskan Malamute,2.0,4.0,5.0,3.0,4.0,1.0,4.0,5.0,2.0,...,4.0,5.0,1.0,5.0,2.0,5.0,6.107824,1161.666667,1155.558843,80%
4,American Bulldog,2.0,3.0,4.0,3.0,3.0,1.0,5.0,1.0,2.0,...,4.0,2.0,1.0,2.0,2.0,3.0,31.551841,979.920635,948.368794,83%
5,American Foxhound,2.0,5.0,5.0,4.0,4.0,1.0,4.0,3.0,5.0,...,3.0,5.0,1.0,4.0,4.0,5.0,9.147743,311.250000,302.102257,75%
6,American Pit Bull Terrier,3.0,4.0,5.0,3.0,4.0,3.0,5.0,4.0,1.0,...,3.0,3.0,1.0,2.0,3.0,4.0,37.339782,1019.047619,981.707837,76%
7,American Staffordshire Terrier,2.0,4.0,4.0,3.0,4.0,2.0,4.0,2.0,2.0,...,3.0,4.0,2.0,2.0,3.0,3.0,28.455378,857.659574,829.204196,60%
8,Australian Cattle Dog,3.0,4.0,5.0,3.0,4.0,2.0,5.0,3.0,4.0,...,3.0,2.0,3.0,4.0,4.0,4.0,20.816758,924.518950,903.702193,79%
9,Australian Kelpie,3.0,3.0,4.0,3.0,5.0,2.0,4.0,3.0,3.0,...,3.0,4.0,2.0,3.0,5.0,4.0,21.614043,786.269841,764.655798,79%


In [ ]:
friendly_attributes = ['All Around Friendliness','Affectionate with Family','Dog Friendly','Friendly Toward Strangers'
                      'Incredibly Kid Friendly Dogs','Potential For Playfulness']
intelligence_attributes = ['Adaptability',' Trainability','Easy To Train','Intelligence']
tolerance_attributes = ['Easy To Groom','Tolerates Being Alone','Tolerates Cold Weather','Tolerates Hot Weather']
negative_attributes = ['Amount Of Shedding','Drooling Potential','Potential For Mouthiness','Potential For Weight Gain',
                      'Prey Drive','Sensitivity Level','Tendency To Bark Or Howl','Wanderlust Potential']

# categories uncombined: 'Adapts Well to Apartment Living','Energy Level','Exercise Needs', 'General Health',
# 'Intensity','Size','Good For Novice Owners'

#cats to get rid of: ' Health Grooming'